In [1]:
from pymodules import sim_functions as sf
from pymodules.sim_functions import b2
import time
import json
from pymodules.ML_params import *

In [2]:
cell = "NMO_00892"

In [3]:
morph_soma = b2.spatialneuron.morphology.Morphology.from_swc_file(f"morphs/{cell}_somas.swc", spherical_soma=True)

In [4]:
morph_d8 = b2.spatialneuron.morphology.Morphology.from_swc_file(f"morphs/{cell}_depth8.swc", spherical_soma=True)
morph_d13 = b2.spatialneuron.morphology.Morphology.from_swc_file(f"morphs/{cell}_depth13.swc", spherical_soma=True)
morph_d14 = b2.spatialneuron.morphology.Morphology.from_swc_file(f"morphs/{cell}_depth14.swc", spherical_soma=True)
morph_full = b2.spatialneuron.morphology.Morphology.from_swc_file(f"morphs/{cell}_full.swc", spherical_soma=True)

In [5]:
α = 4.5 # scaling factor for the soma
β = 1.0 # scaling factor for the dendrites
morphs = [morph_d8, morph_d13, morph_d14, morph_full]

In [6]:
neuron_soma = b2.SpatialNeuron(morphology=morph_soma, model=eqs, Cm=α*Cm, Ri=250*b2.ohm*b2.cm, method='exponential_euler')

In [7]:
len_σ = len(neuron_soma)

In [8]:
Iout = b2.zeros(len(morphs))
rout = b2.zeros(len(morphs))

In [9]:
Ibound_set = [[0.10,0.30]*b2.nA, [0.30,0.35]*b2.nA, [0.35,0.40]*b2.nA, [0.45,0.50]*b2.nA]

In [10]:
rfind = 1.0*b2.Hz
cutoff = 10000

In [11]:
for i,j in enumerate(morphs):
  neuron_d = b2.SpatialNeuron(morphology=morphs[i], model=eqs, Cm=α*Cm, Ri=250*b2.ohm*b2.cm, method='exponential_euler')
  neuron_d[0:len_σ].gL = α*gL0
  neuron_d[len_σ::].gL = β*gL0
  neuron_d.EL = EL0
  neuron_d.Cm[len_σ::] = 0.25*β*Cm
  
  neuron_d[0:len_σ].gf = α*gf0
  neuron_d[0:len_σ].gs = α*gs0; # make only the soma active
  
  tic = time.perf_counter()
  Iout[i], rout[i] = sf.findrate(neuron_d, Ibound_set[i], cutoff, 11000*b2.ms, rfind, vmh = -10.0, Nmax=15, ϵr = 0.02, v0=-10.0*b2.mV, ϵisi=10)
  toc = time.perf_counter()
  print(toc-tic)

35.17465732200071
160.3628649500024
185.9977856609985
244.1271127439977


In [12]:
Iout

array([1.70312500e-10, 3.36196899e-10, 3.80285645e-10, 4.81036377e-10])

In [13]:
rout

array([1.00001  , 1.00001  , 1.700017 , 5.8500585])

In [14]:
jdict = {"Ion": Iout.tolist(), "ron" : rout.tolist()}
with open(f"data/{cell}_morph_onset.json", "w") as json_file:
    json.dump(jdict, json_file)